In [22]:
import torch

a =  torch.tensor([[3, 1, 2, 2],
 [1, 2, 2, 1],
 [3, 5, 3, 4]])
x, indices, cnt = torch.unique(a, return_inverse=True, sorted=False, return_counts=True)

In [23]:
ensemble_attn_mask_per_layer = torch.tensor([[3, 1, 2, 2],
 [1, 2, 2, 1],
 [3, 5, 3, 4]])
# ensemble_attn_mask_per_layer [40, 128, 256, 5] to [N*H * TDST//BLOCK_SIZE_Q, mask_k//BLOCK_SIZE_K * ensemble_model_n]
# ensemble_attn_mask_per_layer = ensemble_attn_mask_per_layer.view(_N_H * TDST_BQ, MASK_K_BK * MODEL_N)
# ensemble_attn_mask_per_layer : [N*H * TDST//BLOCK_SIZE_Q, mask_k//BLOCK_SIZE_K * ensemble_model_n]
unique_x, indices, unique_cnt = torch.unique(ensemble_attn_mask_per_layer, return_inverse=True, sorted=False, return_counts=True)
# indices -= indices.min(dim=1, keepdims=True)[0]

# cnt_x = (unique_x[None, None, :] == ensemble_attn_mask_per_layer[:, :, None]).long().sum(1)
N, K = ensemble_attn_mask_per_layer.shape



In [24]:
unique_x, indices, unique_cnt

(tensor([1, 2, 3, 4, 5]),
 tensor([[2, 0, 1, 1],
         [0, 1, 1, 0],
         [2, 4, 2, 3]]),
 tensor([3, 4, 3, 1, 1]))

In [25]:
import math

cnt_xs = []
chunk_n = 64
for icn in range(int(math.ceil(N / chunk_n))):
    cnt_xs.append(
        (unique_x[None, None, :] == ensemble_attn_mask_per_layer[icn*chunk_n:icn*chunk_n+chunk_n, :, None]).long().sum(1)
    )
cnt_x = torch.cat(cnt_xs, dim=0)


result = torch.full((N, unique_x.shape[0]), 9999999, device=indices.device, dtype=torch.int64)
result = result.scatter_(1, indices, ensemble_attn_mask_per_layer)




In [26]:
cnt_x, result

(tensor([[1, 2, 1, 0, 0],
         [2, 2, 0, 0, 0],
         [0, 0, 2, 1, 1]]),
 tensor([[      1,       2,       3, 9999999, 9999999],
         [      1,       2, 9999999, 9999999, 9999999],
         [9999999, 9999999,       3,       4,       5]]))

In [27]:
# t = result[:, :, None] == unique_x[None, None, :]
# t = t * torch.arange(len(unique_x), device=t.device)[None, None, :]
# t = t.sum(-1)
N, K = result.shape
ts = []
chunk_n = 64
for icn in range(int(math.ceil(N / chunk_n))):
    t = result[icn*chunk_n:icn*chunk_n+chunk_n, :, None] == unique_x[None, None, :]
    t = t * torch.arange(len(unique_x), device=t.device)[None, None, :]
    t = t.sum(-1)
    ts.append(t)
t = torch.cat(ts, dim=0)

# torch.Size([4096, 1280]) torch.Size([64, 1280]) torch.Size([4096, 2094])
# print(result.shape, t.shape, cnt_x.shape)
result_cnt = torch.where(result < 9999999, cnt_x.gather(-1, t), -9999999)

In [28]:
t, result_cnt

(tensor([[0, 1, 2, 0, 0],
         [0, 1, 0, 0, 0],
         [0, 0, 2, 3, 4]]),
 tensor([[       1,        2,        1, -9999999, -9999999],
         [       2,        2, -9999999, -9999999, -9999999],
         [-9999999, -9999999,        2,        1,        1]]))

In [29]:
# ensemble_sorted : [N*H * TDST//BLOCK_SIZE_Q, mask_k//BLOCK_SIZE_K * ensemble_model_n]
# ensemble_cnt_sorted : [N*H * TDST//BLOCK_SIZE_Q, mask_k//BLOCK_SIZE_K * ensemble_model_n]
ensemble_cnt_sorted, indices = torch.sort(result_cnt, dim=-1, descending=True)
ensemble_sorted = result.gather(-1, indices)

In [31]:
ensemble_sorted, ensemble_cnt_sorted, indices

(tensor([[      2,       1,       3, 9999999, 9999999],
         [      1,       2, 9999999, 9999999, 9999999],
         [      3,       4,       5, 9999999, 9999999]]),
 tensor([[       2,        1,        1, -9999999, -9999999],
         [       2,        2, -9999999, -9999999, -9999999],
         [       2,        1,        1, -9999999, -9999999]]),
 tensor([[1, 0, 2, 3, 4],
         [0, 1, 2, 3, 4],
         [2, 3, 4, 0, 1]]))

In [ ]:


'''
ensemble_attn_mask_per_layer
tensor([[1, 1, 3, 3, 3, 5],\
        [3, 3, 4, 4, 4, 4]])
ensemble_cnt_sorted
tensor([[       6,        4,        3,        3,        2,        1,        1,
-9999999, -9999999, -9999999, -9999999, -9999999, -9999999, -9999999,
-9999999, -9999999, -9999999, -9999999, -9999999, -9999999],\
[       6,        4,        3,        3,        2,        2, -9999999,
-9999999, -9999999, -9999999, -9999999, -9999999, -9999999, -9999999,
-9999999, -9999999, -9999999, -9999999, -9999999, -9999999]])
ensemble_sorted
tensor([[   10,     6,     3,     8,     1,     5,    11, 32000, 32000, 32000,
32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000],\
[    5,     4,     7,     9,     6,     3, 32000, 32000, 32000, 32000,
32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000]]))
'''

if ensemble_method_final_inter_thresh == None and ensemble_method_final_bdd_mask_k == 1:
    ensemble_filtered = ensemble_sorted
    ensemble_cnt_filtered = ensemble_cnt_sorted
else:
    mask = ensemble_cnt_sorted >= ensemble_method_final_inter_thresh

    ensemble_filtered = torch.where(mask, ensemble_sorted, torch.tensor(9999999, device=mask.device))
    ensemble_cnt_filtered = torch.where(mask, ensemble_cnt_sorted, torch.tensor(9999999, device=mask.device))
    
## mask_i : where to discard leftovers 
filtered_mask = ensemble_filtered == 9999999
# Determine which columns have all rows as -1
columns_with_all_negative_one = torch.all(filtered_mask, dim=0)

# Get the first index where all rows have -1
nonzero_indices = torch.nonzero(columns_with_all_negative_one, as_tuple=True)

# If there are any such columns, find the first one
if len(nonzero_indices[0]) > 0:
    mask_k_i = nonzero_indices[0][0].item()
    k_final = min(mask_k_i, ensemble_indices_k_size)
else:
    mask_k_i = -1  # If no such index is found
    k_final = ensemble_indices_k_size # CHECK - is it min(ensemble_indices_k_size, ensemble_filtered.shape[1])

ensemble_filtered = ensemble_filtered[:, :k_final] # TODO is this meaningful??
ensemble_cnt_filtered = ensemble_cnt_filtered[:, :k_final]
ensemble_filtered = ensemble_filtered.view(_N_H, TDST_BQ, -1)
ensemble_cnt_filtered = ensemble_cnt_filtered.view(_N_H, TDST_BQ, -1)

In [52]:
import numpy as np

# Create a (3, 5) shaped numpy array
array = np.array([[1, 2, 2, 2, 2],
                  [1, 2, 3, 6, 7],
                  [8, 9, 10, 6, 5]])

# Find the unique values and their counts
unique_values, counts = np.unique(array, return_counts=True)

# Print the results
print("Unique values in the array:", unique_values)
print("Counts of each unique value:", counts)


Unique values in the array: [ 1  2  3  5  6  7  8  9 10]
Counts of each unique value: [2 5 1 1 2 1 1 1 1]


In [79]:
import numpy as np

# Create a (3, 5) shaped numpy array
array = np.array([[1, 2, 2, 2, 5],
                  [1, 2, 5, 6, 7],
                  [8, 9, 10, 6, 5]])

# Find the unique values and their counts
unique_values, counts = np.unique(array, return_counts=True)

unique_values, counts

(array([ 1,  2,  5,  6,  7,  8,  9, 10]), array([2, 4, 3, 2, 1, 1, 1, 1]))

In [54]:
# Sort the unique values by counts in descending order
sorted_indices = np.argsort(counts)[::-1]
sorted_unique_values = unique_values[sorted_indices]
sorted_counts = counts[sorted_indices]

sorted_indices, sorted_unique_values, sorted_counts
# Print the results
# print()
# print("Unique values sorted by counts:", sorted_unique_values)
# print("Counts of each unique value:", sorted_counts)


(array([1, 2, 3, 0, 7, 6, 5, 4]),
 array([ 2,  5,  6,  1, 10,  9,  8,  7]),
 array([4, 3, 2, 2, 1, 1, 1, 1]))

In [55]:
array = np.array([[1, 2, 2, 2, 5],
                  [1, 2, 5, 6, 7],
                  [8, 9, 10, 6, 5]])

mask = array > 3

result = np.where(mask, array, 99999)
result

array([[99999, 99999, 99999, 99999,     5],
       [99999, 99999,     5,     6,     7],
       [    8,     9,    10,     6,     5]])

In [61]:
array = np.array([[[1, 2, 2, 2, 5],
                  [1, 2, 5, 6, 7],
                  [8, 9, 10, 6, 5]], [[1, 1, 1, 2, 3],
                  [1, 2, 3, 6, 8],
                  [8, 9, 10, 4, 6]]])

array[0,0:2].flatten(), array[0,0:2]

(array([1, 2, 2, 2, 5, 1, 2, 5, 6, 7]),
 array([[1, 2, 2, 2, 5],
        [1, 2, 5, 6, 7]]))

In [62]:
f = array[0,0:2].flatten()
unique_values, counts = np.unique(f, return_counts=True)
unique_values, counts

(array([1, 2, 5, 6, 7]), array([2, 4, 2, 1, 1]))

In [65]:
len(unique_values)

5

In [63]:
f = array[0,0:2]
unique_values, counts = np.unique(f, return_counts=True)
unique_values, counts

(array([1, 2, 5, 6, 7]), array([2, 4, 2, 1, 1]))

In [72]:
len(array[0][0])

5

In [73]:
torch.full((3,5),9999)

tensor([[9999, 9999, 9999, 9999, 9999],
        [9999, 9999, 9999, 9999, 9999],
        [9999, 9999, 9999, 9999, 9999]])

In [78]:
type(np.full((3,5),9999)[0][0])

numpy.int64

In [84]:
array = np.array([[[1, 2, 2, 2, 5],
                  [1, 2, 5, 6, 7],
                  [8, 9, 10, 6, 5]], [[1, 1, 1, 2, 3],
                  [1, 2, 3, 6, 8],
                  [8, 9, 10, 4, 6]]])

array[0, 0:2, :], (array[0, 0:2, :]).flatten()

(array([[1, 2, 2, 2, 5],
        [1, 2, 5, 6, 7]]),
 array([1, 2, 2, 2, 5, 1, 2, 5, 6, 7]))

In [86]:
unique_values, unique_cnt = np.unique(array[0, 0:2, :].flatten(), return_counts=True)

unique_values, unique_cnt

(array([1, 2, 5, 6, 7]), array([2, 4, 2, 1, 1]))

In [92]:

sorted_indices = np.argsort(-unique_cnt)
# sorted_indices = np.argsort(unique_cnt)[::-1]
sorted_unique_values = unique_values[sorted_indices]
sorted_cnt = unique_cnt[sorted_indices]

# sorted_unique_values = unique_values
# sorted_cnt = unique_cnt

# assert ensemble_method_final_inter_thresh != None
mask = sorted_cnt >= 2

ensemble_filtered = np.where(mask, sorted_unique_values, 9999999)
ensemble_cnt_filtered = np.where(mask, sorted_cnt, 9999999)

In [93]:
sorted_indices, sorted_unique_values, sorted_cnt

(array([1, 0, 2, 3, 4]), array([2, 1, 5, 6, 7]), array([4, 2, 2, 1, 1]))

In [94]:
ensemble_filtered, ensemble_cnt_filtered

(array([      2,       1,       5, 9999999, 9999999]),
 array([      4,       2,       2, 9999999, 9999999]))

In [97]:
ensemble_cnt_filtered = np.where(ensemble_filtered < 9999999, ensemble_cnt_filtered, -9999999)

In [100]:
ensemble_cnt_filtered

array([       4,        2,        2, -9999999, -9999999])

In [102]:
filtered_mask = ensemble_filtered == 9999999

filtered_mask

array([False, False, False,  True,  True])

In [122]:
ensemble_filtered = torch.tensor([[[2,       1,       5, 9999999, 9999999], [3, 5, 9999999, 9999999, 9999999], [4,7, 9, 9999999, 9999999]], [[2,       1,       6, 8, 9999999], [2, 7, 10, 9999999, 9999999], [3, 5, 7, 8, 9999999]]])

In [123]:
ensemble_filtered, ensemble_filtered.shape

(tensor([[[      2,       1,       5, 9999999, 9999999],
          [      3,       5, 9999999, 9999999, 9999999],
          [      4,       7,       9, 9999999, 9999999]],
 
         [[      2,       1,       6,       8, 9999999],
          [      2,       7,      10, 9999999, 9999999],
          [      3,       5,       7,       8, 9999999]]]),
 torch.Size([2, 3, 5]))

In [124]:
filtered_mask = ensemble_filtered.view(6,-1) == 9999999

filtered_mask,filtered_mask.shape

(tensor([[False, False, False,  True,  True],
         [False, False,  True,  True,  True],
         [False, False, False,  True,  True],
         [False, False, False, False,  True],
         [False, False, False,  True,  True],
         [False, False, False, False,  True]]),
 torch.Size([6, 5]))

In [125]:
columns_with_all_negative_one = torch.all(filtered_mask, dim=0)
nonzero_indices = torch.nonzero(columns_with_all_negative_one, as_tuple=True)

columns_with_all_negative_one, columns_with_all_negative_one.shape

(tensor([False, False, False, False,  True]), torch.Size([5]))

In [126]:
columns_with_all_negative_one, nonzero_indices

(tensor([False, False, False, False,  True]), (tensor([4]),))